In [1]:
# Cell 1 - Imports
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import ipywidgets as widgets
from ipywidgets import interact

# Cell 2 - Interactive RC Filter Explorer
def rc_filter(R=10e3, C=0.1e-6):  # R in Ω, C in F. Default 10kΩ + 0.1µF → fc ≈ 159 Hz
    # Transfer function H(s) = 1 / (1 + sRC)
    sys = signal.TransferFunction([1], [R*C, 1])

    plt.figure(figsize=(12, 8))

    # Bode plot
    w, mag, phase = signal.bode(sys, w=np.logspace(0, 6, 1000))
    plt.subplot(2, 2, (1,2))
    plt.semilogx(w, mag, lw=2.5, color='#1f77b4')
    plt.grid(True, which="both", ls="--")
    plt.axvline(1/(2*np.pi*R*C), color='red', ls='--', label=f'f_c = {1/(2*np.pi*R*C)/1e3:.1f} kHz')
    plt.title(f'RC Low-Pass Filter — R = {R/1e3:.1f} kΩ, C = {C*1e9:.1f} nF')
    plt.ylabel('Magnitude (dB)')
    plt.legend()

    plt.subplot(2, 2, 3)
    plt.semilogx(w, phase, lw=2.5, color='#ff7f0e')
    plt.grid(True, which="both", ls="--")
    plt.axvline(1/(2*np.pi*R*C), color='red', ls='--')
    plt.ylabel('Phase (degrees)')
    plt.xlabel('Frequency (Hz)')

    # Step response
    t, y = signal.step(sys, T=np.linspace(0, 5*R*C, 1000))
    plt.subplot(2, 2, 4)
    plt.plot(t*1e3, y, lw=2.5, color='#2ca02c')
    plt.grid(True)
    plt.title('Step Response (τ = RC = {:.1f} ms)'.format(R*C*1e3))
    plt.xlabel('Time (ms)')
    plt.ylabel('Output Voltage (V)')
    plt.tight_layout()
    plt.show()

interact(rc_filter,
         R=widgets.FloatSlider(min=1e3, max=100e3, step=1e3, value=10e3, description='Resistance (Ω)'),
         C=widgets.FloatSlider(min=1e-9, max=10e-6, step=10e-9, value=0.1e-6, description='Capacitance (F)',
                               style={'description_width': 'initial'},
                               readout_format='.1e'));

interactive(children=(FloatSlider(value=10000.0, description='Resistance (Ω)', max=100000.0, min=1000.0, step=…

In [3]:
# Fixed & Improved Spring-Mass-Damper System (Colab-ready)
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets
from ipywidgets import interact

def spring_mass_damper(m=1.0, k=100.0, c=1.0, x0=1.0, v0=0.0):
    wn = np.sqrt(k/m)                    # natural frequency
    zeta = c / (2 * np.sqrt(m*k))        # damping ratio

    def equations(state, t):
        x, v = state
        return [v, -(c/m)*v - (k/m)*x]

    t = np.linspace(0, 25, 2500)
    sol = odeint(equations, [x0, v0], t)
    x = sol[:, 0]

    # Determine damping type for the title
    if zeta < 0.95:
        damping_type = "Underdamped → Oscillates"
        color = '#d62728'
    elif zeta <= 1.05:
        damping_type = "Critically damped → Fastest return, no oscillation"
        color = '#2ca02c'
    else:
        damping_type = "Overdamped → Slow return, no oscillation"
        color = '#1f77b4'

    plt.figure(figsize=(12, 6))
    plt.plot(t, x, lw=2.8, color=color)
    plt.axhline(0, color='k', lw=1, alpha=0.5)
    plt.grid(True, alpha=0.4)
    plt.title(f"Spring-Mass-Damper System\n"
              f"m = {m:.1f} kg │ k = {k:.1f} N/m │ c = {c:.2f} Ns/m │ "
              f"ωₙ = {wn:.2f} rad/s │ ζ = {zeta:.3f}\n"
              f"→ {damping_type}",
              fontsize=14, pad=20)
    plt.xlabel("Time (s)", fontsize=12)
    plt.ylabel("Displacement (m)", fontsize=12)
    plt.xlim(0, t[-1])
    plt.ylim(-1.5*abs(x0), 1.5*abs(x0))
    plt.tight_layout()
    plt.show()

# Interactive sliders
interact(spring_mass_damper,
         m=widgets.FloatSlider(min=0.2, max=10.0, step=0.2, value=1.0, description='Mass (kg)'),
         k=widgets.FloatSlider(min=10, max=500, step=10, value=100, description='Stiffness k (N/m)'),
         c=widgets.FloatLogSlider(min=-2, max=1.7, step=0.05, base=10, value=1.0,
                                  description='Damping c (Ns/m)'),
         x0=widgets.FloatSlider(min=-2.0, max=2.0, step=0.1, value=1.0, description='Initial disp. (m)'),
         v0=widgets.FloatSlider(min=-10.0, max=10.0, step=0.5, value=0.0, description='Initial vel. (m/s)'));

interactive(children=(FloatSlider(value=1.0, description='Mass (kg)', max=10.0, min=0.2, step=0.2), FloatSlide…

In [5]:
# Project 3: Two-Body Orbital Mechanics – Fixed & Ready for Colab
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets
from ipywidgets import interact

# Constants
GM = 3.986004418e14  # Earth's gravitational parameter (m³/s²)
Re = 6371000         # Earth radius in meters

def orbit_sim(altitude=400000, velocity=7780, eccentricity_factor=0.0, show_earth=True, steps=20000):
    r0 = Re + altitude                           # initial distance from center of Earth
    state0 = [r0, 0, 0, velocity * (1 + eccentricity_factor)]  # [x, y, vx, vy]

    def equations(state, t):
        x, y, vx, vy = state
        r = np.sqrt(x**2 + y**2)
        ax = -GM * x / r**3
        ay = -GM * y / r**3
        return [vx, vy, ax, ay]

    t_max = 200 * 60  # simulate up to 200 minutes
    t = np.linspace(0, t_max, steps)
    sol = odeint(equations, state0, t)
    x, y = sol[:, 0], sol[:, 1]

    # Estimate orbital period
    dist_to_start = np.sqrt((x - r0)**2 + y**2)
    try:
        period_idx = np.argmin(dist_to_start[1000:]) + 1000
        period_min = t[period_idx] / 60
        period_text = f"{period_min:.1f} min"
    except:
        period_text = "Incomplete orbit"

    plt.figure(figsize=(10, 10))
    if show_earth:
        earth = plt.Circle((0, 0), Re/1000, color='#1f77b4', alpha=0.5, label='Earth')
        plt.gca().add_patch(earth)

    plt.plot(x/1000, y/1000, lw=2, color='#ff7f0e')
    plt.plot(x[0]/1000, y[0]/1000, 'go', markersize=8, label=f'Start ({altitude/1000:.0f} km)')
    plt.plot(x[-1]/1000, y[-1]/1000, 'ro', markersize=6, label='End')

    plt.title(f"Orbit | Altitude {altitude/1000:.0f} km | Velocity {velocity:.0f} m/s | Period ≈ {period_text}")
    plt.xlabel("x (km)")
    plt.ylabel("y (km)")
    plt.axis('equal')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()

# Interactive sliders – now with correct closing parenthesis!
interact(orbit_sim,
         altitude=widgets.FloatSlider(min=200000, max=2000000, step=50000, value=400000, description='Altitude (m)'),
         velocity=widgets.FloatSlider(min=5000, max=8500, step=10, value=7780, description='Velocity (m/s)'),
         eccentricity_factor=widgets.FloatSlider(min=0.0, max=0.95, step=0.01, value=0.0, description='Eccentricity factor'),
         show_earth=widgets.Checkbox(value=True, description='Show Earth'),
         steps=widgets.IntSlider(min=5000, max=30000, step=1000, value=20000, description='Smoothness'));

interactive(children=(FloatSlider(value=400000.0, description='Altitude (m)', max=2000000.0, min=200000.0, ste…